In [62]:
# auto reload notebook deps
%reload_ext autoreload
%autoreload 2

In [63]:
import json
import pandas as pd

DATASETS = {
    "CentralParkNYC": {
        "input_location": "./data/tweet_objects/CentralParkNYC/2021-01-27-2021-02-06.json",
        "output_location": "./data/mstream_datasets/CentralParkNYC-2021-01-27-2021-02-06.json",
        "spike_start": "Feb 1, 2021, 17:00",
        "spike_end": "Feb 3, 2021, 04:00",
    },
    "united": {
        "input_location": "./data/tweet_objects/united/2020-12-05-2020-12-15.json",
        "output_location": "./data/mstream_datasets/united-2020-12-05-2020-12-15.json",
        "spike_start": "Dec 12, 2020, 05:00",
        "spike_end": "Dec 13, 2020, 06:00",
    }
}

selected_dataset = DATASETS["CentralParkNYC"]
labeled_spike_start = pd.to_datetime(selected_dataset["spike_start"]).tz_localize("UTC")
labeled_spike_end = pd.to_datetime(selected_dataset["spike_end"]).tz_localize("UTC")

In [64]:
from utils.dataset import create_tweet_df

with open(selected_dataset["input_location"], "r") as f:
    raw_dataset = json.load(f)

df_tweets = create_tweet_df(raw_dataset["tweets"])

# Label tweets
df_tweets["is_anomaly"] = df_tweets.created_at.apply(
    lambda x: 0 if (x < labeled_spike_start or x > labeled_spike_end) else 1
)

df_tweets.head()

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,replied_to,retweeted,quoted,is_anomaly
34642,1336444683875016705,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:56:57+00:00,[],"[DavidCornDC, united]",None,779922519285301248,NaN,1336361619107049472,NaN,0
34641,1336444690640465920,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:56:58+00:00,[],"[DavidCornDC, united]",None,161003217,NaN,1336361619107049472,NaN,0
34640,1336444721556668416,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:57:06+00:00,[],"[DavidCornDC, united]",None,161003217,NaN,1336361619857809408,NaN,0
34639,1336444729723064320,"RT @DavidCornDC: Worse, @United, your flight a...",2020-12-08 22:57:08+00:00,[],"[DavidCornDC, united]",None,794448786,NaN,1336361619857809408,NaN,0
34638,1336444732554113024,"RT @DavidCornDC: Hey @United, a family member ...",2020-12-08 22:57:08+00:00,[],"[DavidCornDC, united]",None,1165791085626396672,NaN,1336361619107049472,NaN,0


In [65]:
# Export 
df_tweets.to_json(
    selected_dataset["output_location"],
    orient="records"
)

In [66]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34643 entries, 34642 to 0
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   id                   34643 non-null  object             
 1   text                 34643 non-null  object             
 2   created_at           34643 non-null  datetime64[ns, UTC]
 3   hashtags             34643 non-null  object             
 4   mentions             34643 non-null  object             
 5   in_reply_to_user_id  15238 non-null  object             
 6   user_id              34643 non-null  object             
 7   replied_to           13384 non-null  object             
 8   retweeted            16921 non-null  object             
 9   quoted               2340 non-null   object             
 10  is_anomaly           34643 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(9)
memory usage: 4.2+ MB
